In [1]:
!pip install barbar

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import numpy as np
from torchvision.datasets import CelebA
from torchvision import transforms, models

import matplotlib.pyplot as plt

import time, copy

from barbar import Bar

from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

# Mount drive first

In [3]:
ls drive/MyDrive/metrics_iclr_2022/celeba/

identity_CelebA.txt   list_attr_celeba.txt  list_eval_partition.txt
img_align_celeba.zip  list_bbox_celeba.txt  list_landmarks_align_celeba.txt


In [4]:
pwd

'/content'

In [5]:
cp -r drive/MyDrive/metrics_iclr_2022/celeba/ .

In [6]:
cd celeba/

/content/celeba


In [7]:
ls

identity_CelebA.txt   list_attr_celeba.txt  list_eval_partition.txt
img_align_celeba.zip  list_bbox_celeba.txt  list_landmarks_align_celeba.txt


In [8]:
!unzip -qq img_align_celeba.zip

In [9]:
cd ..

/content


In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_preprocessing = transforms.Compose([
    transforms.RandomSizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize,
])

val_preprocessing = transforms.Compose([
    transforms.RandomSizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize,
])

def hair_transform(features):
  """
  creates feature of length 5 (Black_hair, Blond_Hair, Brown_Hair, Gray_Hair, None)
  """

  org_features = features[[8, 9, 11, 17]]
  # no hair features set none true
  hair_count = torch.count_nonzero(org_features)
  if hair_count > 1 :
    return torch.tensor(4)
  else:
    return torch.argmax(org_features, 0)

def hair_filter(features):
  """
  creates feature of length 5 (Black_hair, Blond_Hair, Brown_Hair, Gray_Hair, None)
  """

  return torch.count_nonzero(features[[8, 9, 11, 17]])



/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:917: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  warnings.warn("The use of the transforms.RandomSizedCrop transform is deprecated, " +


In [11]:
celeba = CelebA(root='.', split='all', transform=train_preprocessing, target_transform=hair_transform)
#valid_celeba = CelebA(root='.', split='valid', transform=val_preprocessing, target_transform=hair_transform)
#test_celeba = CelebA(root='.', split='test', transform=val_preprocessing, target_transform=hair_transform)


In [12]:
labels = celeba.attr
label_indices = np.arange(len(labels))
keep = torch.stack([hair_filter(t) for t in labels], axis=-1)
train_idx, val_idx = train_test_split(
                  label_indices[keep > 0], 
                  random_state=42,
                  shuffle=True,
                  train_size=0.8)

train_dataset = Subset(celeba, train_idx)
val_dataset = Subset(celeba, val_idx)

val_dataset.dataset.transform = val_preprocessing

In [13]:

dataloaders = {
    "train": torch.utils.data.DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=4),
    "val": torch.utils.data.DataLoader(val_dataset, batch_size=512, shuffle=False, num_workers=1),
    #"test": torch.utils.data.DataLoader(test_celeba, batch_size=512, shuffle=False, num_workers=1),
}

dataset_sizes = {
    "train": len(train_dataset),
    "val": len(val_dataset),
    #"test": len(test_celeba),
}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [14]:
from torch.cuda.amp import autocast, GradScaler
scaler = GradScaler()

In [15]:
def train_model(model, criterion, dataloaders, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in Bar(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                  with autocast():
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        scaler.scale(loss).backward()
                        #loss.backward()
                        scaler.step(optimizer)
                        scaler.update()
                        #optimizer.step()

                _, preds = torch.max(outputs.detach(), 1)
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [16]:
model = models.resnet34(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [ ]:
model = train_model(model, criterion, dataloaders, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


   512/100476: [>...............................] - ETA 0.0s

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


100476/100476: [===============================>] - ETA 6.9s
train Loss: 0.6619 Acc: 0.7618
25120/25120: [===============================>] - ETA 2.2s
val Loss: 0.4593 Acc: 0.8415

Epoch 1/24
----------
100476/100476: [===============================>] - ETA 6.5s
train Loss: 0.4245 Acc: 0.8541
25120/25120: [===============================>] - ETA 2.1s
val Loss: 0.4043 Acc: 0.8626

Epoch 2/24
----------
100476/100476: [===============================>] - ETA 6.5s
train Loss: 0.3959 Acc: 0.8626
25120/25120: [===============================>] - ETA 2.1s
val Loss: 0.3912 Acc: 0.8656

Epoch 3/24
----------
100476/100476: [===============================>] - ETA 6.1s
train Loss: 0.3796 Acc: 0.8691
25120/25120: [===============================>] - ETA 2.1s
val Loss: 0.3750 Acc: 0.8719

Epoch 4/24
----------
100476/100476: [===============================>] - ETA 6.3s
train Loss: 0.3669 Acc: 0.8738
 1536/25120: [=>..............................] - ETA 101.7s

In [ ]:


class_names = ['Black_hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair', "combo"]

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) 

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['train']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                print(preds[j])
                ax.set_title('predicted: {}, Actual: {}'.format(class_names[preds[j]], class_names[labels[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
visualize_model(model)